In [ ]:
# http://www.capo.gov.cy/capo/capo.nsf/All/B3C8BE19E2B8C3A8C2257FC6003EB33A/$file/BENEFICIARIES_2014_FINAL_v3.xlsx
# http://www.capo.gov.cy/capo/CAPO.nsf/All/D81AAD936F05B416C2257FC6003F385F/$file/%CE%A0%CE%BB%CE%B7%CF%81%CF%89%CE%BC%CE%AD%CF%82%20%CE%B4%CE%B9%CE%BA%CE%B1%CE%B9%CE%BF%CF%8D%CF%87%CF%89%CE%BD%202015%20-%2031-5-16%20.xlsx
# http://www.capo.gov.cy/capo/capo.nsf/All/BD7C94C6B96F36D8C2257E5A00385E12/$$file/BENEFICIARIES_2014_FINAL_v3.xlsx

In [9]:
!wget -nc -O cy_2016_raw.xlsx 'http://www.capo.gov.cy/capo/capo.nsf/05082a1738ff5899c225776f002ea4a8/AD063B69CE4D0D06C2258131002AA09F/$file/FINAL%20%CE%A0%CE%BB%CE%B7%CF%81%CF%89%CE%BC%CE%AD%CF%82%20%CE%94%CE%B9%CE%BA%CE%B1%CE%B9%CE%BF%CF%8D%CF%87%CF%89%CE%BD%202016-%2031.5.17.xlsx'

--2023-11-03 07:39:21--  http://www.capo.gov.cy/capo/capo.nsf/05082a1738ff5899c225776f002ea4a8/AD063B69CE4D0D06C2258131002AA09F/$file/FINAL%20%CE%A0%CE%BB%CE%B7%CF%81%CF%89%CE%BC%CE%AD%CF%82%20%CE%94%CE%B9%CE%BA%CE%B1%CE%B9%CE%BF%CF%8D%CF%87%CF%89%CE%BD%202016-%2031.5.17.xlsx
Auflösen des Hostnamens www.capo.gov.cy (www.capo.gov.cy) … 212.31.118.152
Verbindungsaufbau zu www.capo.gov.cy (www.capo.gov.cy)|212.31.118.152|:80 … verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 302 Found
Platz: https://www.capo.gov.cy/capo/capo.nsf/05082a1738ff5899c225776f002ea4a8/AD063B69CE4D0D06C2258131002AA09F/$file/FINAL%20%CE%A0%CE%BB%CE%B7%CF%81%CF%89%CE%BC%CE%AD%CF%82%20%CE%94%CE%B9%CE%BA%CE%B1%CE%B9%CE%BF%CF%8D%CF%87%CF%89%CE%BD%202016-%2031.5.17.xlsx [folgend]
--2023-11-03 07:39:21--  https://www.capo.gov.cy/capo/capo.nsf/05082a1738ff5899c225776f002ea4a8/AD063B69CE4D0D06C2258131002AA09F/$file/FINAL%20%CE%A0%CE%BB%CE%B7%CF%81%CF%89%CE%BC%CE%AD%CF%82%20%CE%94%CE%B9%CE%BA%CE%B1%CE%B9%C

In [ ]:
import pandas as pd

In [ ]:
YEAR = 2016

In [ ]:
excel = pd.ExcelFile('cy_%s_raw.xlsx' % YEAR)
excel.sheet_names

In [ ]:
SHEET_NAME = 'Πληρωμές Δικαιούχων' # means 'Beneficiary Payments'

In [ ]:
df = pd.read_excel('cy_%s_raw.xlsx' % YEAR, sheetname=SHEET_NAME)
original_columns = list(df.columns)
df.head()

In [ ]:
original_columns

In [ ]:
translations = [
    'recipient_name',
    'application_code',
    'EAGF',
    'EAFRD',
    'total',
    'scheme_code',
    'scheme_code_short',
    'scheme_description',
    'recipient_postcode',
    'recipient_location'
]

In [ ]:
df = df.rename(columns=dict(zip(original_columns, translations)))
df['year'] = YEAR
df = df.drop(columns=['scheme_code'])
df['scheme'] = df.apply(lambda x: '%s - %s' % (x['scheme_code_short'], x['scheme_description']), 1)
df.head()

In [ ]:
# df_2014 = pd.read_excel('BENEFICIARIES_2014_FINAL_v3.xlsx')
# original_columns = list(df_2014.columns)
# translations = ('CODE MEASURE (ABB CODE)',
#                'CODE EPEXIGISIS MEASURE',
#                'MEASURE', 'CODE APPLICANT', 'NAME / TRADE NAME', 'EAGF', 'EAFRD', 'POSTAL CODE', 'COMMUNITY / DISTRICT')
# df_2014 = df_2014.rename(columns=dict(zip(original_columns, translations)))
# df_2014 = df_2014.rename(columns={
#     'COMMUNITY / DISTRICT': 'recipient_location',
#     'POSTAL CODE': 'recipient_postcode',
#     'NAME / TRADE NAME': 'recipient_name',
#     'CODE EPEXIGISIS MEASURE': 'scheme'
# }).drop(['CODE MEASURE (ABB CODE)', 'MEASURE'], 1)
# df_2014['year'] = 2014
# df_2014.head()

In [ ]:
# There only ever is one value filled EAGF or EAFRD
((df[['EAGF', 'EAFRD']].fillna(0.0) == 0.0).sum(1) > 1).any()

In [ ]:
from slugify import slugify

df['country'] = 'CY'
df['currency'] = 'EUR'

df['recipient_id'] = df.apply(lambda x: 'CY-%s' % x['application_code'] if pd.notnull(x['application_code']) else
                                        'CY-%s-%s' % (x['recipient_postcode'], slugify(
                                            '' if pd.isnull(x['recipient_name']) else x['recipient_name'])), 1)

df['amount'] = df.apply(lambda x: x[['EAGF', 'EAFRD']].sum(), 1)
df = df.drop(['application_code', 'EAFRD', 'EAGF'], 1)
df.head()

In [ ]:
df.to_csv('cy_%s.csv.gz' % YEAR, index=False, encoding='utf-8', compression='gzip')